# Assignment
In this assignment you will learn about the `rasterio` library and interact with Sentinel scenes in an AWS cloud bucket. You will also learn about Cloud-Optimized Geotiffs (COGs) and the Spatio-Temporal Asset Catalog (STAC) specification.

## Background
You have learned how to use the `geopandas` library to interact with geospatial vector data. The next step is to interact with raster data. The primary library that we use for raster is `rasterio`.Due to the nature of raster datasets and, in large part because of continous earth observation from satellites, a number of standards and technologies have built around optimizing finding raster datasets and utilizing subsets of raster datasets without necessitating the download of copious amounts of imagery that end users have no interest in. The main concepts we will talk about here are Cloud-Optimized Geotiffs, or COGs, windowed reads, and the Spatio-Temporal Asset Catalog (STAC) standard for searching for satellite imagery and other spatio-temporal data.

Background reading:
- COGs
  - https://developers.planet.com/docs/planetschool/an-introduction-to-cloud-optimized-geotiffs-cogs-part-1-overview/
- Rasterio
  - Windows Reading: https://rasterio.readthedocs.io/en/latest/topics/windowed-rw.html
- STAC
  - Intro to STAC: https://stacspec.org/en/tutorials/intro-to-stac/
  - About STAC: https://stacspec.org/en/about/stac-spec/ 

## Housekeeping
### Learning Objectives
1) Become familiar with python libraries: `rasterio` and `satsearch`.
2) Learn about Cloud-Optimized Geotiffs (COGs)
3) Learn about Spatio-Temporal Asset Catalogs (STACs)

### Administrative Needs
1) Configure this Notebook to use the new conda environment for its kernel



## Setting up the python environment
Open this file (`assignment.ipynb`) in the Editor

A dialog should pop-up asking you to install recommended `Python` and `Jupyter` extensions.

![vscode-install-extensions.png](./media/vscode-install-extensions.png)

Click on the kernel selector at the top right hand corner of the Jupyter Notebook in the `Editor` panel.

![select-kernel.png](./media/select-kernel.png)

and select the `geo-python-38` item.

![select-geo-python-kernel.png](./media/select-geo-python-kernel.png)

## Walking through this notebook

Congratulations! You now have a Jupyter Notebook running in a cloud environment with geospatially-enabled python. The rest of this tutorial will focus on the content (Finally!)

This is adapted from Simon Planzer's tutorial on [NDVI Time Series with COGs](https://www.simonplanzer.com/articles/cog-ndvi-part1/). 

## Part 1 - Finding imagery with STAC

The first step is to import our needed libraries and initialize some global variables.

In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import rasterio
from rasterio.plot import show, reshape_as_image
from pyproj import Transformer
from satsearch import Search

%matplotlib inline

# Sentinel-2 STAC API
url = "https://earth-search.aws.element84.com/v0/"

This function will search the STAC catalog for `sentinel-s2-l2a-cogs`. For more about what data is returned from a STAC query, see [STAC specification](https://stacspec.org/en/about/stac-spec/). Essentially it is a list of STAC items and possibly additional STAC catalogs, allowing a hierarchical grouping of STAC items. Here we are only interested in the `items` which will contain a `href`, or a link to where can find the data. Since the collection we are searching _only_ contains COGs we will be able to read the data from within a small Jupyter notebook.

There is a python library, `satsearch` that handles a lot of this for us, and we are going to wrap that in a helper function that takes a bounding box, date range, and cloud tolerance.

In [ ]:
def image_search(bbox, date_range, scene_cloud_tolerance):
    """
    Using SatSearch find all Sentinel-2 images
    that meet our criteria
    """
    
    search = Search(
        bbox=bbox,
        datetime=date_range,
        query={
            "eo:cloud_cover": {"lt": scene_cloud_tolerance}
        },  
        collections=["sentinel-s2-l2a-cogs"],
        url=url,
    )

    return search.items()

Next we define some constants for our search. You could modify these to your own choosing if you want. The settings below are for a couple of summer scenes in the mountains east of Tucson, AZ.

In [ ]:
# Bounding Box delineating the spatial extent for NDVI mapping
bbox = [-110.74772571639987, 32.270431012618026, -110.70996215904789, 32.29386169894274]

# The date range for mapping NDVI overtime
date_range = "2021-07-05/2021-08-02"

This is where we search the catalog using our helper function. The `items.summary()` prints a summary of what we found in our search. Note that STAC does not include data; just metadata about the scenes, including where we can access the actual image data. For a more thorough discussion and walkthrough of STAC catalog items, see https://github.com/sat-utils/sat-stac/blob/master/tutorial-2.ipynb. 

First, let's search for imagery in our area and time range, then print a summary. 

In [ ]:
items = image_search(bbox, date_range, 5)
items.summary()

`items.summary()` gives a brief description of what we have; namely, the names of the items. Let's look at an individual item more closely.

## Deliverable 1: `stac-items.png`
Take a screenshot of this page showing the output of `items.summary()` and save it as `stac-items.png`. You can upload it directly in the `Explorer` panel. Right click on the space below the file list and select "Upload"

![upload-screenshot.png](./media/upload-screenshot.png)

Pick one item from the catalog and look at some of its properties. This will show its bounding box.

In [ ]:
item = items[0]
print(item.bbox)

Look at a few more properties of the item.

In [ ]:

print(item.geometry)
print("-")
print(item.datetime)
print("-")
print(item.date)
print("-")
print(item.assets)


Those `assets` are a little hard to read. Let's see what it is and how we can interact with it better

In [ ]:
type(item.assets)

Since it's a `dict` that means it has `.keys()` method:

In [ ]:
item.assets.keys()

Looks like satellite image bands and some additional rasters like a thumbnail, an overview, and a visual product plus water vapor clouds, etc. Let's see what we have for the `visual` asset.

In [ ]:
item.assets["visual"]

Notice that `href` entry which contains an obvious link to data. For fun, you can copy that URL and open it in QGIS with `Layer` -> `Add Layer` -> `Add Raster Layer` and enter the URL of the scene.

Meanwhile, let's see what other metadata this item has:

In [ ]:
for key in item.properties:
    print('%s: %s' % (key, item.properties[key]))

## Part 2 - Reading COGs

Ok, that's enough messing around. Let's go play with the data. Performing windowed reads is not the most straightforward so we will use a helper function to abstract away some of the details about the window so we can just pass in the image url and the bounding box. This function will make a windowed read from the remote COG based on some url and return the image pixel data as a `numpy.ndarray`.

In [ ]:
transform_window = None
def range_request(image_url, bbox):
    """
    Request and read just the required pixels from the COG
    """
    
    with rasterio.open(image_url) as src:
        coord_transformer = Transformer.from_crs("epsg:4326", src.crs)
        # calculate pixels to be streamed from the COG
        coord_upper_left = coord_transformer.transform(bbox[3], bbox[0])
        coord_lower_right = coord_transformer.transform(bbox[1], bbox[2])
        pixel_upper_left = src.index(coord_upper_left[0], coord_upper_left[1])
        pixel_lower_right = src.index(coord_lower_right[0], coord_lower_right[1])
         
                
        # request only the bytes in the window
        window = rasterio.windows.Window.from_slices(
            (pixel_upper_left[0], pixel_lower_right[0]),
            (pixel_upper_left[1], pixel_lower_right[1]),
        )

        # The affine transform - This will allow us to 
        # translate pixels coordiantes back to geospatial coordiantes
        transform_window = rasterio.windows.transform(window,src.transform)
        
        bands = 1
        if "TCI" in image_url: # aka True Colour Image aka RGB
            bands = [1, 2, 3]

        subset = src.read(bands, window=window)
        return(subset, transform_window)

Put it into action using the first item in the list we got back from STAC

In [ ]:
img = items[0].asset("red")["href"]
img_sub, transform_window = range_request(img, bbox)
print(type(img_sub))


To show that it really is pixel data, look at the `img_sub` variable:

In [ ]:
img_sub

## Deliverable 2: `image-numpy.png`

Take a screenshot of the output above showing your `array([[[...` and name it `image-numpy.png`


## Part 3: Calculate NDVI for scene subsets
This next part is going to calculate NDVI for the windowed area for all the scenes in our `items` list. For each one it will read the `href` dict entry for the `red`, `nir`, and `visual` assets. Then it will download the image subset using the windowed read from `rasterio`, and then it will save the data in a list named `images`. 

_Note: This step can take longer than a minute_

In [ ]:
images= []

for item in items:
    
    # Refs to images
    red = item.asset("red")["href"]
    nir = item.asset("nir")["href"]
    rgb = item.asset("visual")["href"]
    date = item.date.strftime("%m/%d/%Y")

    # Streamed pixels within bbox
    red_subset, transform_window = range_request(red, bbox)
    nir_subset, transform_window = range_request(nir, bbox)
    rgb_subset, transform_window = range_request(rgb, bbox)

    # Calculate NDVI
    ndvi_subset = (nir_subset.astype(float) - red_subset.astype(float)) / (
        nir_subset + red_subset
    )
    
    # Store the data for further processing
    images.append(
        {"date": date, "rgb": rgb_subset, "ndvi": ndvi_subset,'transform_window': transform_window}
    )

# reverse list as to be oldest to newest
images.reverse()

That was fun but what do we have? Let's take a look by using the `show()` function from `rasterio` to view the imagery. Note that this step has often crashed my kernel, in which case you'll need to start over from **Walking through this notebook**. This is the price of using free cloud resources. This would be more stable on a desktop computer.

In [ ]:
### Visualise NDVI and RBG images Side by Side
num_images = len(images)
width, height = 10, num_images * 4.5

fig, subplots = plt.subplots(
    len(images),
    2,
    sharex="col",
    sharey="row",
    figsize=(width, height),
    constrained_layout=True,
)

for plot in zip(subplots, images):
    ax = plot[0]
    image = plot[1]
    rgb_axes = show(image['ndvi'], 
                    transform=image['transform_window'], 
                    ax=ax[0], 
                    alpha=.75,
                    cmap="RdYlGn", 
                    title=image["date"])
    rgb_axes.ticklabel_format(style ='plain') # show full y-coords

    show(image['rgb'], transform=image['transform_window'], ax=ax[1])

## Deliverable 3: `image-plots.png`

Save a screenshot of part of the output of the image plots above and save it as `image-plots.png`

## Part 4 - Clipping rasters by polygons

Let's load up `geopandas` and extract some pixel data based on some polygons I drew of the neighborhood around Agua Caliente Park in Tucson, AZ. Our imagery is in EPSG:32612 so we will automatically reproject the geojson (which is always in lat/long; WGS84) using the `.to_crs()` function og `geopandas`.

In [ ]:
import geopandas 

fields = geopandas.read_file('neighborhood_samples.geojson').to_crs(epsg=32612)

fields.head()
fields.columns

Show the vectors over the image. We will share an axes object or else they will be written in separate graphs or one will overwrite (and erase) the other.

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10))

bbox = [-110.74772571639987, 32.270431012618026, -110.70996215904789, 32.29386169894274]

rgb_axes = show(images[0]['rgb'], 
                transform=images[0]['transform_window'], 
                ax=ax,
                alpha=.75,
                cmap="RdYlGn", 
                title="Veg Types")
fields.boundary.plot(ax=ax, color='skyblue', linewidth=4)
# ax.set_xlim(left=bbox[0], right=bbox[2])
# ax.set_ylim(bottom=bbox[1], top=bbox[3])
rgb_axes.ticklabel_format(style ='plain') # show full y-coords

## Deliverable 4 - `fields.png`

Take a screenshot of the fgure above showing the polygon boundaries over the imagery.

Next we will extract the imagery data within a polygon. Let's define a helper function that will utilize `rasterio`'s `features.geometry_mask()` function to clip rasters by polygons.

In [ ]:
import numpy.ma as ma
from rasterio import features


def mask_ndvi_by_field(image, field_geom):
    """
    returns a numpy mask
    """
    mask = features.geometry_mask(field_geom, 
                                out_shape=(image['ndvi'].shape[0],  image['ndvi'].shape[1]),
                                transform=image['transform_window'], 
                                all_touched=False, 
                                invert=False)
    ndvi_masked = ma.masked_array(image['ndvi'], mask)
    return ndvi_masked

Now that those are defined let's run that function for every feature in the polygon dataset _for every image_. We will iterate over all images and, for each image, iterate over all polygon features. For each of those will will mask the NDVI by field to basically create a new band for each of the images that contains the NDVI only for the polygon areas intersecting the original. 

In [ ]:
for image in images:
    ndvi_fields = {}
    
    for index, row in fields.iterrows():
        field_mask = mask_ndvi_by_field(image, row.geometry)
        ndvi_fields[row.area] = field_mask
        image['ndvi_fields'] = ndvi_fields

Take a look at what you've created, using `images[0]` (the first one in our list):

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
show(images[0]['rgb'], transform=image['transform_window'], ax=ax, alpha=.50)
        
for field_id, field_ndvi in images[0]['ndvi_fields'].items():
    rgb_axes= show(field_ndvi, transform=image['transform_window'],
         ax=ax,
         cmap="RdYlGn", 
         vmin=-1, 
         vmax=1, 
         title=f"NDVI Masked by Fields of Interest ({images[0]['date']})")
    
rgb_axes.ticklabel_format(style ='plain') # show full y-coords

## Part 5: NDVI Time Series 

Finally we are going to calculate the average NDVI within one of the polygons over a lengthy time period.

First we will re-search but for a longer time period. This is largely a copy-paste from earlier in this notebook but expediting the imagery for having NDVI for the full time series.

In [ ]:

# The date range for mapping NDVI overtime
date_range = "2020-01-01/2021-12-31"

long_series_images= []
items = image_search(bbox, date_range, 5)
items.summary()
for item in items:
    
    # Refs to long_series_images
    red = item.asset("red")["href"]
    nir = item.asset("nir")["href"]
    rgb = item.asset("visual")["href"]
    date = item.date.strftime("%m/%d/%Y")

    # Streamed pixels within bbox
    red_subset, transform_window = range_request(red, bbox)
    nir_subset, transform_window = range_request(nir, bbox)
    rgb_subset, transform_window = range_request(rgb, bbox)

    # Calculate NDVI
    ndvi_subset = (nir_subset.astype(float) - red_subset.astype(float)) / (
        nir_subset + red_subset
    )
    
    # Store the data for further processing
    long_series_images.append(
        {"date": date, "rgb": rgb_subset, "ndvi": ndvi_subset,'transform_window': transform_window}
    )

# reverse list as to be oldest to newest
long_series_images.reverse()

Iterate over all the images

In [ ]:
from matplotlib import gridspec
import matplotlib as mpl

# compute the number of rows and columns
n_plots = len(long_series_images)
n_cols = int(np.sqrt(n_plots))
n_rows = int(np.ceil(n_plots / n_cols))


# setup the plot
scale = max(n_cols, n_rows)
fig = plt.figure(figsize=(5 * scale, 5 * scale))
grid = gridspec.GridSpec(n_rows, n_cols, fig, wspace=0.4)


# iterate through each subplot and plot each set of data
for i in range(n_plots):
    ax = fig.add_subplot(grid[i])
    axes_subplot = show(images[i]['rgb'],
                        transform=transform_window, 
                        ax=ax,  
                        alpha=.65, 
                        title = images[i]['date'] )
    
    # plot the field data
    for field_id, field_ndvi in images[i]['ndvi_fields'].items():
        im = show(field_ndvi, transform=transform_window,
             ax=ax,
             cmap="RdYlGn", 
             vmin=-1, 
             vmax=1, 
             )
    axes_subplot.ticklabel_format(style ='plain') # show full y-coords
    

# Add custom color bar
mpl.cm.cool
norm = mpl.colors.Normalize(vmin=-1, vmax=1)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap="RdYlGn"),
             ax=grid.figure.get_axes() , orientation='vertical', label='NDVI')

Finally... This is where we graph the NDVI over one of our polygon features.

In [ ]:
# Use geopandas `loc` to select the field where area name == riparin
riparian = fields.loc[fields['area'] == 'riparian']

fig, ax = plt.subplots(1, figsize=(12,12))
    
rgb_axes = show(images[0]['rgb'], 
                transform=image['transform_window'], 
                ax=ax,
                alpha=.75,
                cmap="RdYlGn", 
                title="Field 6")
fields.boundary.plot(ax=ax, color='skyblue', linewidth=4)
riparian.boundary.plot(ax=ax, color='yellow', linewidth=6)                             
rgb_axes.ticklabel_format(style ='plain') # show full y-coords

## Deliverable 5 - `ndvi-time-series.png`

Save a screenshot of the time series graph above as `ndvi-time-series.png`

## Final Deliverables
Submit an open pull request from the `rasterio` branch to be merged with `master` containing these 5 files. Do not merge your pull request. 
- `stac-items.png`
- `image-numpy.png`
- `image-plots.png`
- `fields.png`
- `ndvi-time-series.png`